vamos a morirnos

In [25]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import geopandas as gdp
from bson.json_util import dumps

In [2]:
df = pd.read_csv('newyork_data.csv', index_col = 0)
df2 = pd.read_csv('mad_data.csv', index_col = 0)
df3 = pd.read_csv('bcn_data.csv', index_col = 0)

In [3]:
def funcionquefunciona (df,ciudad):
    gdf = gdp.GeoDataFrame(df, geometry= gdp.points_from_xy(df.lng, df.lat ))
    print(gdf.columns)
    gdf.columns=['nombre', 'lat','lng','ciudad','distancia','loc','loc2' ]
    
    gdf['loc2']= gdf['loc2'].apply(lambda x:shapely.geometry.mapping(x))
    
    client = MongoClient()
    db = client.GeoProject
    collection = db.create_collection(name = f'{ciudad}')
    collection = db[f'{ciudad}']
    collection.create_index([("loc2", "2dsphere")])
    data = gdf.to_dict(orient='records')
    collection.insert_many(data)
    
    return 'Buenisima'

In [4]:
funcionquefunciona(df2,'mad')

'Buenisima'

In [5]:
funcionquefunciona(df3,'bnc')

'Buenisima'

In [20]:
lista1 = [df,df2,df3]
lista2 = ['ny', 'mad','bcn']

In [21]:
for i,j in zip(lista1,lista2):
    funcionquefunciona(i,j)

Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')
Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')
Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')


#### Pasamos a hacer las geoqueries

Primero nos tenemos que conectar con la base de datos en Mongo

In [4]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoProject")

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoProject')

In [6]:
m = db.get_collection("mad")
b = db.get_collection("bcn")
ny = db.get_collection('ny')

Empezamos con las geoqueries

In [57]:
# estas son las coordenadas que utilizaremos y en el formato que se tienen que usar (long, lat)
coord_ny = [-73.98996, 40.75673]
coord_mad = [2.19763, 41.40080]
coord_bcn = [-3.67097, 40.50222]

In [63]:
query = [{'$geoNear': {'near':[-73.98996, 40.75673],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [64]:
geoloc_ny = ny.aggregate(query)
ny_response_json = json.loads(dumps(geoloc_ny))

In [65]:
ny_response_json[75]

{'_id': {'$oid': '613246bda69b02500131d9d4'},
 'nombre': 'STORY',
 'lat': 40.750865752734846,
 'lng': -73.98927211761475,
 'ciudad': 'New York',
 'distancia': 655,
 'loc': 'POINT (-73.98927211761475 40.75086575273485)',
 'loc2': {'type': 'Point',
  'coordinates': [-73.98927211761475, 40.750865752734846]},
 'distance': 0.6546437953152028}

In [81]:
ny_response_json[75]['distance'],['nombre'],['ciudad']

(0.6546437953152028, ['nombre'], ['ciudad'])

In [43]:
# me genero un dataframe solo con las cosas que me interesan dentro de la respuesta que me ha dado la query
# solo quiero el nombre, la ciudad y la distancia

In [75]:
query2 =[{'$geoNear': {'near':[2.19763, 41.40080],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [76]:
geoloc_mad = m.aggregate(query2)
mad_response_json = json.loads(dumps(geoloc_mad))

In [121]:
mad_response_json[41]

{'_id': {'$oid': '613246bda69b02500131dae8'},
 'nombre': 'Bar Welcome',
 'lat': 40.488635,
 'lng': -3.669062,
 'ciudad': 'Madrid',
 'distancia': 1520,
 'loc': 'POINT (-3.669062 40.488635)',
 'loc2': {'type': 'Point', 'coordinates': [-3.669062, 40.488635]},
 'distance': 502.96841703133106}

In [87]:
coord = [[-73.98996, 40.75673], [2.19763, 41.40080], [-3.67097, 40.50222]]
colecciones = [ny, m , b]

In [162]:
# no funciona bien, me dice que en aggregate un string no tiene esa cualidad, pero le estoy pasando una coleccion ... ?
def geoquery(coordenadas, coleccion): #las coordenadas en formato lista (long y lat)
    query = [{'$geoNear': {'near':f'{coordenadas}',
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]
    
    col = f'{coleccion}'
    geoloc = col.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    
    return response_json

In [92]:
query3 =[{'$geoNear': {'near':[-3.67097, 40.50222],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [96]:
geoloc_bcn = b.aggregate(query3)
bcn_response_json = json.loads(dumps(geoloc_bcn))

In [123]:
bcn_response_json[67]

{'_id': {'$oid': '613246bda69b02500131dbc6'},
 'nombre': 'Starbucks',
 'lat': 41.38190998967705,
 'lng': 2.172113994974859,
 'ciudad': 'Barcelona',
 'distancia': 2993,
 'loc': 'POINT (2.172113994974859 41.38190998967705)',
 'loc2': {'type': 'Point',
  'coordinates': [2.172113994974859, 41.38190998967705]},
 'distance': 500.3295985340484}

Las distancias de 'distance' no tienen sentido en Madrid y Barcelona, en Nueva York coinciden y tienen snetido (comprobado), sin embargo tanto en Madrid como en Barcelona no (comprobado también, respecto a las coordenadas impuestas por mí). 
Por ello pasamos a utilizar la columna de distancia que se generó directamente en el primer json en vez de la de distance que hemos generado con el geoNear.

In [146]:
Barcelona = json_normalize(bcn_response_json)
Barcelona = Barcelona[['nombre', 'ciudad', 'distancia']]
Barcelona = Barcelona[Barcelona['distancia'] < 6000]
Barcelona.head()

,nombre,ciudad,distancia
18,Starbucks Diagonal 593,Barcelona,5753
19,Starbucks On The Go,Barcelona,5523
20,Secrid Pop Up Store,Barcelona,5443
21,What's Up! Berlín,Barcelona,5409
22,Starbucks Diagonal 640,Barcelona,5225


In [147]:
Madrid = json_normalize(mad_response_json)
Madrid = Madrid[['nombre', 'ciudad', 'distancia']]
Madrid = Madrid[Madrid['distancia'] < 6000]
Madrid.head()

,nombre,ciudad,distancia
9,"Edificio Veganova, Alcobendas",Madrid,5476
10,Bar y Restaurante Payger,Alcobendas,5676
11,Bar Sube y Baja,Alcobendas,5293
13,Starbucks Diversia Alcobendas,Alcobendas,4172
15,Eventos Fly Up!,Madrid,4731


In [149]:
Newyork = json_normalize(ny_response_json)
Newyork = Newyork[['nombre', 'ciudad', 'distancia']]
Newyork = Newyork[Newyork['distancia'] < 6000]
Newyork.head()

,nombre,ciudad,distancia
0,Starbucks,New York,55
1,CBOA Pick Up/Drop Off Point #1,New York,69
2,Starbucks,New York,83
3,Applebee's Grill + Bar,New York,96
4,TerraCycle: Green Up Shop,New York,144


In [150]:
Madrid.shape

(44, 3)

In [151]:
Barcelona.shape

(135, 3)

In [152]:
Newyork.shape

(212, 3)

In [156]:
# no funciona, no se por qué
def todataframe (json):
    df = json_normalize(f'{json}')
    df = df[['nombre', 'ciudad', 'distancia']]
    df = df[df['distancia'] < 6000]
    return df